# Youtube Ranking

In [26]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [27]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)
time.sleep(2)

In [28]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [29]:
channel = channel_list[0]
rank = channel.select_one('.rank').get_text().strip()
rank = int(rank)
rank

1

In [30]:
category = channel.select_one('p.category').get_text().strip()
category[1:-1] # 카테고리 앞뒤에 붙은 대괄호 삭제

'음악/댄스/가수'

In [6]:
title = channel.select_one('.subject a').get_text().strip() # subject 하위의 a 태그
title

'BLACKPINK'

In [7]:
subscriber = channel.select_one('.subscriber_cnt').get_text().strip()
view = channel.select_one('.view_cnt').get_text().strip()
video = channel.select_one('.video_cnt').get_text().strip()
subscriber, view, video

('7020만', '214억5709만', '395개')

- 반복문으로 페이지 안의 데이터 수집

In [8]:
lines = []
for channel in channel_list:
    rank = int(channel.select_one('.rank').get_text().strip())
    category = channel.select_one('p.category').get_text().strip()
    title = channel.select_one('.subject a').get_text().strip()
    subscriber = channel.select_one('.subscriber_cnt').get_text().strip()
    view = channel.select_one('.view_cnt').get_text().strip()
    video = channel.select_one('.video_cnt').get_text().strip()
    lines.append([rank, category[1:-1], title, subscriber, view, video])

In [140]:
df = pd.DataFrame(lines, columns=['순위', '카테고리', '채널명', '구독자수', '조회수', '영상수'])
df.head()

,순위,카테고리,채널명,구독자수,조회수,영상수
0,1,음악/댄스/가수,BLACKPINK,7020만,214억5709만,395개
1,2,음악/댄스/가수,HYBE LABELS,6300만,204억9684만,725개
2,3,음악/댄스/가수,BANGTANTV,6150만,140억7794만,"1,646개"
3,4,음악/댄스/가수,SMTOWN,2930만,230억1641만,"3,779개"
4,5,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,223개


In [141]:
df.tail()

,순위,카테고리,채널명,구독자수,조회수,영상수
95,96,키즈/어린이,로미유 스토리[Romiyu Story],417만,10억9524만,434개
96,97,음식/요리/레시피,[Dorothy]도로시,416만,9억9800만,905개
97,98,취미/라이프,JaeYeol ASMR 재열,398만,10억9606만,"1,018개"
98,99,키즈/어린이,CoCosToy 꼬꼬스토이,388만,21억8778만,652개
99,100,BJ/인물/연예인,허팝Heopop,384만,32억6002만,"1,900개"


- 숫자로 바꿔주는 함수

In [17]:
# 구독자수, 조회수, 영상수에 숫자만 남기기
def convert_unit(s):
    s = s.replace('만','0000').replace('억','').replace('개','').replace(',','')
    s = int(s)
    return s

convert_unit('39만18,19만39억39개')

390000181900003939

In [31]:
# 선생님 코드
def convert_unit_s(s):
    s = s.replace('만','0000').replace('억','').replace('개','').replace(',','')
    return f'{int(s):,d}' # 천단위 구분기호 넣기
convert_unit_s('39만18,19만39억39개')

'390,000,181,900,003,939'

In [144]:
# 타자를 좀 덜 쳐보기
import re
def convert_unit1(s):
    s = s.replace('만','0000')
    s = re.sub('[^0-9]','',s)
    s = int(s)
    return s

convert_unit1('39만18,19만39억39개')

390000181900003939

In [145]:
# 데이터 프레임에 적용 1) map() 사용 : 각 열을 따로 불러서
df['구독자수'] = df['구독자수'].map(convert_unit)
df['조회수'] = df['조회수'].map(convert_unit)
df['영상수'] = df['영상수'].map(convert_unit)
df

,순위,카테고리,채널명,구독자수,조회수,영상수
0,1,음악/댄스/가수,BLACKPINK,70200000,21457090000,395
1,2,음악/댄스/가수,HYBE LABELS,63000000,20496840000,725
2,3,음악/댄스/가수,BANGTANTV,61500000,14077940000,1646
3,4,음악/댄스/가수,SMTOWN,29300000,23016410000,3779
4,5,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],26500000,11052880000,223
...,...,...,...,...,...,...
95,96,키즈/어린이,로미유 스토리[Romiyu Story],4170000,1095240000,434
96,97,음식/요리/레시피,[Dorothy]도로시,4160000,998000000,905
97,98,취미/라이프,JaeYeol ASMR 재열,3980000,1096060000,1018
98,99,키즈/어린이,CoCosToy 꼬꼬스토이,3880000,2187780000,652


In [146]:
test_df = pd.DataFrame(lines, columns=['순위', '카테고리', '채널명', '구독자수', '조회수', '영상수'])

In [147]:
# 데이터 프레임에 적용 2) apply() 사용
test_df['구독자수'] = test_df['구독자수'].apply(convert_unit)
test_df['조회수'] = test_df['조회수'].apply(convert_unit)
test_df['영상수'] = test_df['영상수'].apply(convert_unit)
test_df

,순위,카테고리,채널명,구독자수,조회수,영상수
0,1,음악/댄스/가수,BLACKPINK,70200000,21457090000,395
1,2,음악/댄스/가수,HYBE LABELS,63000000,20496840000,725
2,3,음악/댄스/가수,BANGTANTV,61500000,14077940000,1646
3,4,음악/댄스/가수,SMTOWN,29300000,23016410000,3779
4,5,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],26500000,11052880000,223
...,...,...,...,...,...,...
95,96,키즈/어린이,로미유 스토리[Romiyu Story],4170000,1095240000,434
96,97,음식/요리/레시피,[Dorothy]도로시,4160000,998000000,905
97,98,취미/라이프,JaeYeol ASMR 재열,3980000,1096060000,1018
98,99,키즈/어린이,CoCosToy 꼬꼬스토이,3880000,2187780000,652


In [9]:
# test_df['구독자수'] = test_df.구독자수.apply(convert_unit)
# test_df['조회수'] = test_df.조회수.apply(convert_unit)
# test_df['영상수'] = test_df.영상수.apply(convert_unit) # 뒤는 이렇게 불러도 되는구나...
# test_df.head()

In [11]:
# for s in test_df.loc[:, '구독자수':'영상수']:
#     test_df[s] = test_df[s].apply(convert_unit)
# test_df

- 두번째 페이지

In [14]:
# Xpath로 찾아가기 - //*[@id="list-skin"]/nav/span/a[1]
driver.find_element_by_xpath('//*[@id="list-skin"]/nav/span/a[1]').click()
time.sleep(2)

In [15]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [18]:
lines = []
for channel in channel_list:
    rank = int(channel.select_one('.rank').get_text().strip())
    category = channel.select_one('p.category').get_text().strip()
    title = channel.select_one('.subject a').get_text().strip()
    subscriber = channel.select_one('.subscriber_cnt').get_text().strip()
    view = channel.select_one('.view_cnt').get_text().strip()
    video = channel.select_one('.video_cnt').get_text().strip()
    lines.append([rank, category[1:-1], title, convert_unit(subscriber), convert_unit(view), convert_unit(video)])

In [19]:
df = pd.DataFrame(lines, columns=['순위', '카테고리', '채널명', '구독자수', '조회수', '영상수'])
df.head()

,순위,카테고리,채널명,구독자수,조회수,영상수
0,101,TV/방송,워크맨-Workman,3790000,743430000,154
1,103,애완/반려동물,크림히어로즈,3780000,1415780000,1011
2,102,음악/댄스/가수,WINNER,3780000,1204270000,446
3,104,키즈/어린이,Lime Tube[라임튜브],3740000,2646690000,1808
4,105,BJ/인물/연예인,waveya 2011,3730000,1356390000,710


- page 1에서 10까지 크롤링하기

In [32]:
lines = []
for page in range(1, 11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
    driver.get(url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    channel_list = soup.select('.aos-init')

    for channel in channel_list:
        rank = int(channel.select_one('.rank').get_text().strip())
        category = channel.select_one('p.category').get_text().strip()
        title = channel.select_one('.subject a').get_text().strip()
        subscriber = channel.select_one('.subscriber_cnt').get_text().strip()
        view = channel.select_one('.view_cnt').get_text().strip()
        video = channel.select_one('.video_cnt').get_text().strip()
        lines.append([rank, category[1:-1], title, convert_unit_s(subscriber), convert_unit_s(view), convert_unit_s(video)])

In [33]:
df = pd.DataFrame(lines, columns=['순위', '카테고리', '채널명', '구독자수', '조회수', '영상수'])
df.head()

,순위,카테고리,채널명,구독자수,조회수,영상수
0,1,음악/댄스/가수,BLACKPINK,"70,200,000","21,457,090,000",395
1,2,음악/댄스/가수,HYBE LABELS,"63,000,000","20,496,840,000",725
2,3,음악/댄스/가수,BANGTANTV,"61,500,000","14,077,940,000","1,646"
3,4,음악/댄스/가수,SMTOWN,"29,300,000","23,016,410,000","3,779"
4,5,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],"26,500,000","11,052,880,000",223


In [34]:
df.tail()

,순위,카테고리,채널명,구독자수,조회수,영상수
995,996,음악/댄스/가수,Reynah,"550,000","148,880,000",246
996,997,TV/방송,J.Y. Park,"550,000","93,730,000",90
997,998,게임,[ALTUBE] 김성현TV,"550,000","422,100,000","2,121"
998,1000,게임,방앗간 비둘기,"550,000","356,900,000","4,381"
999,999,게임,빅헤드,"550,000","326,600,000","3,540"


In [35]:
df.to_csv('유튜브랭킹.csv', index=False)
driver.close()